In [1]:
from app.api import *
import subprocess
import python_speech_features

10-18 12:52:11 Model para number = 1.02


In [2]:
filename = "/home/jjjj/Downloads/Telegram Desktop/video_2023-10-17_09-46-14 (2).mp4"
min_track = 10
num_failed_det = 15
min_face_size = 20

In [3]:
scene = scene_detect(filename)
faces, fps = inference_video(filename)


Detected: 0 | Progress: 100%|██████████| 576/576 [00:00<00:00, 1368.53frames/s]


In [4]:
all_tracks = []
for shot in scene:
    if shot[1].frame_num - shot[0].frame_num >= min_track:  # Discard the shot frames less than min_track frames
        all_tracks.extend(
            track_shot(
                faces[shot[0].frame_num:shot[1].frame_num],
                num_failed_det, min_track, min_face_size
            )
        )

In [5]:
def load_audio(file: str, s: float, e: float, sr: int):

    # This launches a subprocess to decode audio while down-mixing
    # and resampling as necessary.  Requires the ffmpeg CLI in PATH.
    # fmt: off
    cmd = [
        "ffmpeg",
        "-nostdin",
        "-threads", "0",
        "-i", file,
        "-f", "s16le",
        "-ac", "1",
        "-acodec", "pcm_s16le",
        "-ar", str(sr),
        "-vn",
        "-ss", str(round(s, 3)),
        "-to", str(round(e, 3)),
        "-"
    ]
    # fmt: on
    try:
        out = subprocess.run(cmd, capture_output=True, check=True).stdout
    except subprocess.CalledProcessError as e:
        raise RuntimeError(f"Failed to load audio: {e.stderr.decode()}") from e

    return np.frombuffer(out, np.int16).flatten().astype(np.float32) / 32768.0

In [6]:
track = all_tracks[0]

In [7]:
track

{'frame': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]),
 'bbox': array([[163.48434448, 211.29476929, 266.38827515, 331.43270874],
        [176.96244812, 205.71656799, 277.39749146, 324.09585571],
        [180.88363647, 205.54849243, 281.63470459, 321.5836792 ],
        [182.00500488, 206.63066101, 281.31344604, 320.02853394],
        [183.07113647, 211.02459717, 282.25946045, 326.88204956],
        [180.05834961, 221.88616943, 277.13726807, 337.33145142],
        [175.06637573, 234.88972473, 269.41574097, 349.5475769 ],
        [174.87684631, 242.21629333, 268.92330933, 356.12353516],
        [176.14678955, 249.0785675 , 268.88220215, 363.39135742],
        [171.86953735, 254.82148743, 267.03143311, 366.7180481 ],
        [170.84684753, 257.86080933, 266.84957886, 368.71578979],
        [172.28907776, 259.17971802

In [8]:
start, end = track["frame"][0], track["frame"][-1]

In [9]:
sample_rate = 16000

In [10]:
audio = load_audio(filename, start / fps, end / fps, sample_rate)
audioFeature = python_speech_features.mfcc(audio, sample_rate, numcep=13, winlen=0.025, winstep=0.010)


In [11]:
audioFeature

array([[-3.60436534e+01,  0.00000000e+00,  0.00000000e+00, ...,
         9.64028692e-15,  0.00000000e+00,  0.00000000e+00],
       [-3.60436534e+01,  0.00000000e+00,  0.00000000e+00, ...,
         9.64028692e-15,  0.00000000e+00,  0.00000000e+00],
       [-3.60436534e+01,  0.00000000e+00,  0.00000000e+00, ...,
         9.64028692e-15,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 4.95320604e-02, -1.64071056e+00, -3.50602840e+00, ...,
        -5.17540217e+00, -9.11730647e+00,  2.97705278e+00],
       [-2.10130152e-02,  2.95154933e-01, -1.86630310e+00, ...,
        -5.67386433e+00, -4.51968343e+00,  3.17187692e+00],
       [-5.63171961e-01,  1.86943760e+00, -1.18358153e+00, ...,
        -5.92517366e+00,  3.00121979e+00,  5.79589609e+00]])

In [12]:
len(all_tracks)

25

In [16]:
track.keys()

dict_keys(['frame', 'bbox'])

In [ ]:
track["faces"] = []

In [18]:
cap = cv2.VideoCapture(filename)

for track_id in range(len(all_tracks)):
    
    start, end = all_tracks[track_id]["frame"][0], all_tracks[track_id]["frame"][-1]
    all_tracks[track_id]["faces"] = []
    
    for i in range(start, end + 1):
        cap.set(1, i)
        ret, frame = cap.read()
        face = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face = cv2.resize(face, (224, 224))
        face = face[int(112 - (112 / 2)):int(112 + (112 / 2)), int(112 - (112 / 2)):int(112 + (112 / 2))]
        all_tracks[track_id]["faces"].append(face)


In [21]:
len(all_tracks)

25

In [23]:
all_tracks[0].keys()

dict_keys(['frame', 'bbox', 'faces'])